In [1]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_name = 'republic-project'
repo_dir = os.path.split(os.getcwd())[0].split(repo_name)[0] + repo_name
print("adding project dir to path:", repo_dir)
if repo_dir not in sys.path:
    sys.path = [repo_dir] + sys.path
else:
    sys.path.remove(repo_dir)
    sys.path = [repo_dir] + sys.path
    


adding project dir to path: /Users/marijnkoolen/Code/Huygens/republic-project


In [2]:
from flair.models import SequenceTagger

flair_dir = f'{repo_dir}/data/embeddings/flair_embeddings/'

# load the model you trained
model_dir = '../../data/embeddings/flair_embeddings/resources/taggers/ner-tbd-DAT-train_1.0-without_flair-mini_batch_size_8/'
train_size = 1.0
layer_name = 'DAT'
use_flair = True
with_flair = 'with_flair' if use_flair else 'without_flair'

model = {}

layers = ['DAT', 'HOE', 'LOC', 'ORG', 'COM', 'PER', 'RES', 'NAM']

for layer_name in ['single_layer']:
    model_dir = f'{flair_dir}/resources/taggers/ner-tbd-{layer_name}-train_{train_size}-{with_flair}-mini_batch_size_8'

    model[layer_name] = SequenceTagger.load(f'{model_dir}/final-model.pt')



2023-06-16 18:24:05,804 SequenceTagger predicts: Dictionary with 35 tags: O, S-HOE, B-HOE, E-HOE, I-HOE, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-ORG, B-ORG, E-ORG, I-ORG, S-DAT, B-DAT, E-DAT, I-DAT, S-RES, B-RES, E-RES, I-RES, S-COM, B-COM, E-COM, I-COM, S-NAM, B-NAM, E-NAM, I-NAM, <START>, <STOP>


In [3]:
from IPython.display import HTML as html_print


In [44]:
from collections import defaultdict

import republic.tag.entities as ent_tag

docs = defaultdict(list)

for layer in model:
    layer_test_file = ent_tag.get_layer_test_file(layer, repo_dir)
    docs[layer] = [doc for doc in ent_tag.read_test_file(layer_test_file)]
    print(layer, len(docs[layer]))

single_layer 163


In [5]:

LAYER_COLOR = {
    'DAT': 'blue',
    'HOE': 'red',
    'LOC': 'green',
    'ORG': 'purple',
    'PER': 'SlateBlue',
    'RES': 'DodgerBlue',
    'COM': 'MediumSeaGreen',
    'NAM': 'Orange'
}



layer_name = 'single_layer'
texts = []

for di in range(len(docs[layer_name])):
    doc = docs[layer][di]
    sentence = Sentence(doc['text'])
    model[layer].predict(sentence)
    tagged_position = get_tagged_positions(sentence)
    #print('\nTAGGED_POSITION:', tagged_position, '\n')
    #print('\nTAG_POSITION:', doc['tag_position'], '\n')
    text = highlight_tagged_text(sentence)
    texts.append('<h2>Tagger</h2>' + text)
    #print('TAGGED:', text)
    text = highlight_tagged_text(sentence)
    #print('TAG:', text)
    texts.append('<h2>Manual</h2>' + text)
    if di > 2:
        break
    
html_print('\n\n'.join(texts))

In [7]:
# source: https://nos.nl/artikel/2472853-weer-biden-tegen-trump-voor-de-amerikanen-hoeft-dat-niet-zo
raw_texts = [
    """Na maanden om de hete brij heen draaien heeft Joe Biden officieel bekendgemaakt wat iedereen al had verwacht: hij gaat op voor een tweede termijn als president.""",
    """Net als in 2020 zet Biden zichzelf neer als de beste kandidaat die het kan opnemen tegen de beweging van Donald Trump, die hij omschrijft als extremisten die verworven vrijheden willen afpakken van de gewone Amerikaan.""",
    """Als Donald Trump de Republikeinse presidentskandidaat wordt, kan de race om het Witte Huis volgend jaar zomaar een revanchewedstrijd worden. Maar de kiezer is minder enthousiast over dit déjà vu-scenario. In een nieuwe peiling van NBC News zegt 70 procent dat Biden zich niet opnieuw verkiesbaar moet stellen (de peiling werd gehouden voor de officiële bekendmaking) en 60 procent vindt dat Trump niet nog een keer presidentskandidaat moet zijn. Dit betekent niet dat de kiezer niet op hen zou stemmen, maar de metaalmoeheid neemt toe."""
]




tagged_texts = []

for text in raw_texts:
    sentence = Sentence(text)
    model[layer].predict(sentence)
    tagged_position = get_tagged_positions(sentence)
    text = highlight_tagged_text(sentence)
    tagged_texts.append(text)
    
html_print('\n\n'.join(tagged_texts))


In [8]:
raw_texts = [
    """Ik vond dit boek op een kamer van een cliënt die ik verzorgde, ik was geïntrigeerd over de titel en het liet me niet meer los. Daarom heb ik het boek tijdens mijn vakantie meegenomen. Tot nu toe heb ik daar geen spijt van gehad.""",
    """Het is het dagboek van Hendrik Groen, Hendrik woont in een verzorgingshuis in Nederland, hij heeft een paar ouderdomskwaaltjes waarover hij regelmatig een overleg heeft met zijn huisarts en geriater. Het leuke aan deze gesprekjes vond ik dat hij ze steeds op stang jaagt met de vraag om dé pil. Zonder dat hij er voorlopig gebruik van wil maken want er zijn nog genoeg leuke dingen in het leven volgens Hendrik.""",
    """Hij besluit om het anders te gaan doen. Hij verzamelt een groepje medestanders en richt de club oud-maar-nog-niet-dood op. Samen organiseren ze om de beurt een uitje. Een kookworkshop, golfen, of naar een 3D film. Iets wat de activiteitenbegeleiders van het huis niet direct aan zouden denken. Maar ook iets wat de kok en de directrice van het huis niet echt kunnen waarderen maar omdat de reglementen niet openbaar worden gemaakt niet verboden kunnen worden."""
]

tagged_texts = []

for text in raw_texts:
    sentence = Sentence(text)
    model[layer].predict(sentence)
    tagged_position = get_tagged_positions(sentence)
    text = highlight_tagged_text(sentence)
    tagged_texts.append(text)
    
html_print('\n\n'.join(tagged_texts))


In [8]:
text = """Ontfangen een missive vanden Commissaris Johan reimst, geschreven tot Paris den 3e. deses & daernevens drie bijlagen, houdende poincten van conlideratie, raeckende t' stuck vande marine; Waerop gedelibereert sijnde hebben d'Heeren van Hollandt vande voors missive ende bijlagen versocht copie die haer E. wort geaccordeert om den haren breder gecommuniceert te worden, & sal oock aen d‛andere provincien des begerende gelijcke copie gegeven worden. o: ordinaris ambassadeur in Engelandt geschre ven tot Londen den tweeden deses houdende advertentie, ende onder anderen eenige poincten van — consideratie. Waerop gedelibereert sijnde, hebben de Provincien daervan versocht copie om in den haren breder gecommuniceert te worden, die haer E wert geaccordeert."""

#text = """Ende sal mt gene voors: is bij aenschrijvinge aenden meerwelgem: heer Prince willen van Nassau bekent gemaeckt worden, met authorisatie, om bij soo verre de noot sulcx mochte comen te vereijfschen, de voors: Compagnie te peerde, ende te voet alle, off soodanigh getal van deselve, als hij sal oordeelen noodigh te hebben, bij sichte ontbieden, ende deselve provisionelijck te rengeren onder soodanige heeft-officieren, als sijn vorsel: Dooch:, ten meesten — dienste van den Lande geraden vinden zal. Des dat soo wanneer de voors: — tweede Lijste in 't gehtel bij nader deliberatie sal wesen gearresteert, deselve Compagnien wederm gerengeert ende gebruijckt sullen werden onder soodanige regemecten, ende hooft officieren, als ij deselve Lijste breeder sal wesen — geexpresseert. Ende wert den Raed van State versocht de officieren van de bovengeroerde Compagnien aen te schrijven, datse haer gereet sullen hebben te houden — om aenstonts opt ontfangen van de ordre van den meergem: heer prince willem te connen Marcheren, als mede soodanigen ordre te stellen ende die) voorsieninge doen, ten eijnde dat de voors: Compagnien tot de voors: — expeditie gebruijckt werdende in haer bandeliers met cruijt, loot, ende lonten, uijt de respective magasijnen behoorlijck voorsien mogen werden: — mitsgaders de officieren te voet aente schrijven, datse hare Compagnien hebben te formeren in twee deelen musquettiers, en een deel piecken, ende in cas — deselve daer toe eenige wapenen van Nooden, ofte te verwisselen moesten — hebben, deselve onder hare recipissen te lichten uijt de respective magasijnen, mits dat deselve op haer wederkeeren behoorlijck werden gerestitueert, dat oock de crancke, ende mmipotente soldaten, ende die althans op Zee — geemploijeert mochten sijn, uijt de andere Compagnien in deselve guarnisoen „nen, ofte daerontrent gelegen werden verwisselt, ende geremplaceert. Des wert den voorn: Raed van State, oock wijders versocht, alle de officieren — alhier in den hage sijnde, achte doen seggen, datse haer naer hunne respective Guarnisoenen sullen hebben te begeven, oock de Gouverneurs ende Commandeurs te ordonneren, datse alle de absente officieren sullen hebben te beschrijven, om in hare respective Guarnisoenen te verschijnen, ende aldaer te — verblijven tot nader ordre. Ende sullen alle de voorn: Compagnien, voor soo veel sij sullen comen te ageren, naer de expeditie wederleeren in hunne respective guarnisoenen, daer sij uijt getrocken sijn, Eijntlijck is verstaen, dat aede Provincie van Stadt Groningen ende Ommelanden, gelijck oock — aen den Gouverneur van Wesel, ende Commandeurs van Lieroort, Bellinge, wolde, ende Langhackerschans, geschreven sal worden, dat deselve sigh exactelijck willen informeren, ende doen informeren, wat bij den Bisschep van Minister — ende sijne Troupes soude mogen aengevangen, ende ondernomen werden: — gevende daer van telckens spoedige advertentie aen haer Ho:Mo: mitsgas aen den meergem: heer Prince willen van Nassau, Wijders is verstaen, dat de voors: tweede Lijste, voor soo veel aengaet de resterende Compagnien — te peerde ende te voet, daerinne gementioneert, op dewelcke noch niet en is geresolveert, andermael gestelt sal werden in handen vande opgem: heeren hare Ho:Mo: Gedepe., ende Gecome uijt den Raed van State, op nader te visiteren, examineren, ende daer van rapport te doen. Ende sal van alle het gene voors: is Extract gesonden werden aenden meerwelgem: heer Prince Willem van Nassau, om te strecken tot sijne narichtnnge, met recommandatie, omme Lomme, ende daer het behoort conrespondentie te houden, ende doen houden, oock trachten soo veel mogelijck te vernemen, wat daecontrent omgaet, ende hare ho:Mo: van tijt tot tijt advertentie daervan geven. sullende alle de depesches hier uijt resulterende werden — geexpedieert"""

tagged_text = tag_text(text, model)
highlighted_text = highlight_tagged_text(tagged_text)

html_print(highlighted_text)


## Entities and Formulas

In [9]:
from republic.elastic.republic_elasticsearch import initialize_es

rep_es = initialize_es()

In [9]:
import re


def separate_special_tags(text, special_tags):
    for tag in special_tags:
        text = text.replace(f'><{tag}', f'> <{tag}')
        text = text.replace(f'{tag}><', f'{tag}> <')
    return text


def merge_tags(text):
    special_tags = ['DAT', 'RES']
    text = text.replace('> <', '><')
    text = separate_special_tags(text, special_tags)
    merged_text = text
    replacements = []
    for m in re.finditer(r'((<[A-Z]+>)+)', text):
        tag_string = m.group(1)
        if tag_string[1:-1] in special_tags:
            merge_tag = tag_string
        elif '<DAT>' in tag_string:
            raise ValueError('Date merged with other tags:', tag_string)
        elif '<COM>' in tag_string:
            merge_tag = '<COM>'
        elif tag_string.startswith('<HOE>'):
            merge_tag = '<HOE>'
        elif tag_string.startswith('<ORG>'):
            merge_tag = '<ORG>'
        elif tag_string.startswith('<LOC>'):
            merge_tag = '<LOC>'
        elif '<HOE>' in tag_string:
            merge_tag = '<HOE>'
        elif '<PER>' in tag_string:
            merge_tag = '<PER>'
        else:
            print('UNEXPECTED TAG:', tag_string)
        replacements.append((m.start(), tag_string, merge_tag))
    for start, tag_string, merge_tag in replacements[::-1]:
        end = start + len(tag_string)
        merged_text = merged_text[:start] + merge_tag + merged_text[end:]
    return merged_text



In [ ]:
query = {}
resolutions = rep_es.retrieve_resolutions_by_query(None, size=150)

for resolution in resolutions:
    res_text = '\n'.join([para.text for para in resolution.paragraphs])
    text = res_text
    target_words = ['gedelibereerdt', 'gedelibereert', 'deliberatie']
    if any([target_word in res_text for target_word in target_words]):
        continue
    if 'op geen resolutie is gevallen' in res_text.lower():
        continue
    #print(res_text)
    sentence = Sentence(res_text)
    model[layer].predict(sentence)
    tagged_positions = get_tagged_positions(sentence)
    for tagged_position in tagged_positions:
        start, end = tagged_position
        tag_type = tagged_positions[tagged_position]
        #print(start, end, tag_type, text[start:end])
        text = text[:start] + f'<{tag_type}>' + text[end:]
        #print(text)
        #print('\n')
    merged_text = merge_tags(text)
    print(merged_text)
    print('\n\n')


In [58]:
import gzip


def make_inventory_query(inventory_num):
    return {
        'bool': {
            'must': [
                {'match': {'metadata.inventory_num': inventory_num}}
            ]
        }
    }


inv_nums = [i for i in range(3775, 3865)]

for inv_num in inv_nums:
    out_file = f'../../data/resolutions/ner_tagged/resolutions-{inv_num}.tsv.gz'
    query = make_inventory_query(inv_num)
    with gzip.open(out_file, 'wt') as fh:
        for ri, resolution in enumerate(rep_es.scroll_resolutions_by_query(query, scroll='5m')):
            if (ri+1) % 1000 == 0:
                print(inv_num, ri+1, resolution.id)
            res_text = '\n'.join([para.text for para in resolution.paragraphs])
            text = res_text
            sentence = Sentence(res_text)
            model[layer].predict(sentence)
            tagged_positions = get_tagged_positions(sentence)
            for tagged_position in tagged_positions:
                start, end = tagged_position
                tag_type = tagged_positions[tagged_position]
                text = text[:start] + f'<{tag_type}>' + text[start:end] + f'</{tag_type}>' + text[end:]
            fh.write(f"{resolution.id}\t{text}\n")
        print(inv_num, ri+1, resolution.id)


total hits: {'value': 4150, 'relation': 'eq'} 	hits per scroll: 10
3771 1000 session-1716-05-16-ordinaris-num-1-attendance_list
3771 2000 session-1716-05-11-ordinaris-num-1-resolution-7
3771 3000 session-1716-05-22-ordinaris-num-1-resolution-25
3771 4000 session-1716-01-10-ordinaris-num-1-resolution-5
3771 4150 session-1716-12-28-ordinaris-num-1-resolution-15
total hits: {'value': 4084, 'relation': 'eq'} 	hits per scroll: 10
3772 1000 session-1717-07-16-ordinaris-num-1-resolution-7
3772 2000 session-1717-06-15-ordinaris-num-1-resolution-16
3772 3000 session-1717-11-17-ordinaris-num-1-resolution-5
3772 4000 session-1717-09-17-ordinaris-num-1-attendance_list
3772 4084 session-1717-07-01-ordinaris-num-1-attendance_list
total hits: {'value': 3862, 'relation': 'eq'} 	hits per scroll: 10
3773 1000 session-1718-03-31-ordinaris-num-1-resolution-1
3773 2000 session-1718-09-13-ordinaris-num-1-attendance_list
3773 3000 session-1718-03-24-ordinaris-num-1-resolution-10
3773 3862 session-1718-07-25-

KeyboardInterrupt: 

In [65]:
from republic.elastic.republic_elasticsearch import initialize_es

rep_es = initialize_es()


def make_query(inv_num):
    return {
        'bool': {
            'must': [
                {'match': {'metadata.type': 'resolution'}},
                {'match': {'metadata.inventory_num': inv_num}}
            ]
        }
    }


for inv_num in range(3767, 3768):
    query = make_query(inv_num)
    print('inv_num:', inv_num)
    res_dir = '../../data/paragraphs/loghi/'
    res_file = os.path.join(res_dir, f"resolution-paragraphs-Loghi-{inv_num}.tsv.gz")
    with gzip.open(res_file, 'wt') as fh:
        for hit in rep_es.scroll_hits(rep_es.es_anno, query, index='full_resolutions'):
            doc = hit['_source']
            res_year = doc['metadata']['session_year']
            prop_type = doc['metadata']['proposition_type']
            for para in doc['paragraphs']:
                if doc['id'] == 'session-1712-07-05-ordinaris-num-1-resolution-12':
                    print(doc['id'], prop_type)
                    print((f"{res_year}\t{doc['id']}\t{prop_type}\t{para['text']}\n"))
                fh.write(f"{res_year}\t{doc['id']}\t{prop_type}\t{para['text']}\n")


inv_num: 3767
total hits: {'value': 5413, 'relation': 'eq'} 	hits per scroll: 100
session-1712-07-05-ordinaris-num-1-resolution-12 missive
1712	session-1712-07-05-ordinaris-num-1-resolution-12	missive	ONtfangen een Missive van den Resident vanden Bosch, geschreven te Hamburgh den eersten deser loopende maendt, en daer nevens een Bylage, houdende advertentie, ende onder anderen, dat den Raedt aldaer door hare Gedeputeerden hem dien avondt hadden laten brengen de Extracten van Brieven nevens de voorschreve Missive gevoeght, uyt Bergen ende Housum geschreven, met versoeck, dat hy aen haer Hoogh Mogende wilde voordragen de schade, die de Commercie van haer ende van haer Hoogh Mogende Ingezetenen daer door quamen te lyden; dat sy versochten dat haer Hoogh Mogende in der selver faveur aen het Deensche Hof geliefden te schryven, ende aen hare Ministers te gelasten, der selver goede officien aen het voorschreve Hof te doen, ten eynde de aengehoudene Schepen, waer van'er reets drie geconfisquee

In [66]:
import gzip
from collections import namedtuple

def read_inventory_resolutions(inv_num):

    res_dir = '../../data/paragraphs/loghi/'
    res_file = os.path.join(res_dir, f"resolution-paragraphs-Loghi-{inv_num}.tsv.gz")
    with gzip.open(res_file, 'rt') as fh:
        prev_id = None
        prev_year = None
        prev_type = None
        paras = []
        for line in fh:
            res_year, res_id, prop_type, para = line.strip().split('\t')
            if prev_id and prev_id != res_id:
                res_text = '  '.join(paras)
                #print(prev_id, len(paras))
                yield Resolution(year=prev_year, id=prev_id, prop_type=prev_type, text=res_text)
                paras = []
            paras.append(para)
            prev_id = res_id
            prev_year = res_year
            prev_type = prop_type
        if len(paras) > 0:
            yield Resolution(year=prev_year, id=prev_id, prop_type=prev_type, text=res_text)
    return None


Resolution = namedtuple('Resolution', 'year id prop_type text')

for ri, res in enumerate(read_inventory_resolutions(3775)):
    continue
print(ri)

2802


In [67]:
for inv_num in [3767]:
    for ri, res in enumerate(read_inventory_resolutions(inv_num)):
        if res.id == 'session-1712-07-05-ordinaris-num-1-resolution-12':
            print('\t', res.prop_type)
            print(res.text)


	 missive
ONtfangen een Missive van den Resident vanden Bosch, geschreven te Hamburgh den eersten deser loopende maendt, en daer nevens een Bylage, houdende advertentie, ende onder anderen, dat den Raedt aldaer door hare Gedeputeerden hem dien avondt hadden laten brengen de Extracten van Brieven nevens de voorschreve Missive gevoeght, uyt Bergen ende Housum geschreven, met versoeck, dat hy aen haer Hoogh Mogende wilde voordragen de schade, die de Commercie van haer ende van haer Hoogh Mogende Ingezetenen daer door quamen te lyden; dat sy versochten dat haer Hoogh Mogende in der selver faveur aen het Deensche Hof geliefden te schryven, ende aen hare Ministers te gelasten, der selver goede officien aen het voorschreve Hof te doen, ten eynde de aengehoudene Schepen, waer van'er reets drie geconfisqueert waren, mochten vrygegeven, ende in het toekomende de Commercie weder vry gedreven werden, dat hy aengenomen hadde het voorschreve aen haer Hoogh Mogende te sullen voordragen, om dat hy see

In [41]:
start = time.time()
time.sleep(2)
step = time.time()
start, step, step - start, f"{step - start: >.0f} seconds"

(1687211367.391179, 1687211369.3960419, 2.0048627853393555, '2 seconds')

In [28]:
f"{1.231: >6.2}"

'   1.2'

In [68]:
import gzip
import time


inv_nums = [i for i in range(3760, 3865)]

start_time = time.time()

for inv_num in inv_nums:
    out_file = f'../../data/resolutions/ner_tagged/resolutions-{inv_num}.tsv.gz'
    with gzip.open(out_file, 'wt') as fh:
        for ri, res in enumerate(read_inventory_resolutions(inv_num)):
            if (ri+1) % 1000 == 0:
                step_time = time.time()
                took = step_time - start_time
                print(inv_num, ri+1, res.id, f"{took: >.0f} seconds")
            text = res.text
            res_text = res.text.replace('\n', '    ').replace('\t', '    ')
            sentence = Sentence(res_text)
            model[layer].predict(sentence)
            tagged_positions = get_tagged_positions(sentence)
            for tagged_position in tagged_positions:
                start, end = tagged_position
                tag_type = tagged_positions[tagged_position]
                text = text[:start] + f'<{tag_type}>' + text[start:end] + f'</{tag_type}>' + text[end:]
            fh.write(f"{res.id}\t{text}\n")
        step_time = time.time()
        took = step_time - start_time
        print(inv_num, ri+1, res.id, f"{took: >.0f} seconds")


3760 1000 session-1705-09-12-ordinaris-num-1-resolution-12 179 seconds
3760 2000 session-1705-04-29-ordinaris-num-1-resolution-5 355 seconds
3760 3000 session-1705-06-15-ordinaris-num-1-resolution-21 529 seconds
3760 4000 session-1705-07-10-ordinaris-num-1-resolution-12 706 seconds
3760 5000 session-1705-06-19-ordinaris-num-1-resolution-29 877 seconds
3760 6000 session-1705-06-27-ordinaris-num-1-resolution-17 1061 seconds
3760 7000 session-1705-05-15-ordinaris-num-1-resolution-15 1230 seconds
3760 7073 session-1705-04-02-ordinaris-num-1-resolution-2 1243 seconds
3761 1000 session-1706-07-26-ordinaris-num-1-resolution-7 1406 seconds
3761 2000 session-1706-04-06-ordinaris-num-1-resolution-2 1593 seconds
3761 3000 session-1706-09-20-ordinaris-num-1-resolution-17 1757 seconds
3761 4000 session-1706-01-23-ordinaris-num-1-resolution-21 1929 seconds
3761 5000 session-1706-08-09-ordinaris-num-1-resolution-4 2113 seconds
3761 6000 session-1706-01-30-ordinaris-num-1-resolution-1 2305 seconds
376

## Splitting glued resolutions

In [ ]:
import glob
import gzip
import os


def read_tagged_resolutions(tagged_res_dir, split_dir):
    tagged_res_files = glob.glob(os.path.join(tagged_res_dir, '*.gz'))

    for tagged_res_file in tagged_res_files:
        tagged_dir, tagged_file = os.path.split(tagged_res_file)
        #print([tagged_dir, tagged_file])
        split_res_file = os.path.join(split_dir, tagged_file)
        #print(split_res_file)
        with gzip.open(tagged_res_file, 'rt') as fh:
            data = fh.read()
            lines = ['session-17' + line for line in data.split('session-17') if line != '']
            print(len(lines), tagged_file)
            with gzip.open(split_res_file, 'wt') as fh_out:
                for line in lines:
                    fh_out.write(f"{line}\n")
    return None



tagged_res_dir = '../../data/resolutions/ner_tagged/'
split_dir = '../../data/resolutions/ner_tagged_split/'

read_tagged_resolutions(tagged_res_dir, split_dir)